In [ ]:
import numpy as np
from math import sqrt
num_pats = 100
sq = 1;                         # Size of square
r = sq/sqrt(2*np.pi);              # Radius of circle so it's half area of square
xcent = 0.5
ycent = 0.5                    # Centre of circle
xs = xcent*np.ones((num_pats, 1))
ys = ycent*np.ones((num_pats, 1))
rng = np.random.Generator(np.random.PCG64(12345))
# rng = np.random.Generator(np.random.PCG64())
a = np.concatenate((xs,ys), axis=1) + 0.5-rng.random((num_pats, 2))
bmat = ((a[:, 0, np.newaxis]-xcent)**2 + (a[:, 1, np.newaxis]-ycent)**2) > r**2
bmat = np.concatenate((bmat, 1-bmat), axis=1)

In [ ]:
%matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.ion()

figure, axes = plt.subplots()
draw_circle = plt.Circle((xcent, ycent), r, fill=False)
input_area = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor="g", facecolor='none')
# x = list(a[0][0:3])
# y = list(a[1][0:3])
plotted_x = []
plotted_y = []
draw_points = axes.scatter(plotted_x, plotted_y)

axes.set_aspect(1)
axes.add_patch(draw_circle)
axes.add_patch(input_area)
axes.add_artist(draw_points)

plt.title('Circle')

def reset_plot():
    global figure, axes, draw_circle, input_area, plotted_x, plotted_y, draw_points
    figure, axes = plt.subplots()
    draw_circle = plt.Circle((xcent, ycent), r, fill=False)
    input_area = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor="g", facecolor='none')
    plotted_x = []
    plotted_y = []
    draw_points = axes.scatter(plotted_x, plotted_y)

    axes.set_aspect(1)
    axes.add_patch(draw_circle)
    axes.add_patch(input_area)
    axes.add_artist(draw_points)

    plt.title('Circle')
    plt.draw()

rects = {}
def update_rect(index, bottom_left, height, width, in_class):
    edge_color = "b"
    if in_class:
        edge_color = "r"
    if height < 0.0001 and width < 0.0001:
        height = 0.01
        width = 0.01
    rect = patches.Rectangle(bottom_left, width, height, linewidth=1, edgecolor=edge_color, facecolor='none')
    if index in rects:
        rects[index].remove()
    rects[index] = rect
    axes.add_artist(rect)
    figure.canvas.draw_idle()
    plt.pause(0.01)

def add_point(x,y):
    plotted_x.append(x)
    plotted_y.append(y)
    draw_points.set_offsets(np.c_[plotted_x, plotted_y])
    figure.canvas.draw_idle()
    plt.pause(0.01)

In [ ]:
# Replication of section VI. Simulation Circle-in-the-square, from Carpenter et al., 1992

# 0 1 == _in_
# 1 0 == _out_
from fuzzy_artmap import FuzzyArtMap
from collections import Counter
import complement_encode
reset_plot()
fuzzy_artmap = FuzzyArtMap(4,20)
for i in range(num_pats):
    test_input = a[np.newaxis, i, :]
    ground_truth = bmat[np.newaxis, i, :]
    complement_encoded_input = complement_encode.complement_encode(test_input)
    add_point(test_input[0][0], test_input[0][1])
    fuzzy_artmap.train(complement_encoded_input, ground_truth)
out_boxes = fuzzy_artmap.weight_ab[:, 0] > fuzzy_artmap.weight_ab[:, 1]
print(np.count_nonzero(fuzzy_artmap.weight_a[:, 0] < 1, 0)) # overall number of categories

for j in range(fuzzy_artmap.weight_a.shape[0]):
    is_out_box = out_boxes[j]
    bottom_left = (fuzzy_artmap.weight_a[j,0], fuzzy_artmap.weight_a[j,1])
    width = (1-fuzzy_artmap.weight_a[j,2]) - fuzzy_artmap.weight_a[j,0]
    height =(1-fuzzy_artmap.weight_a[j,3]) - fuzzy_artmap.weight_a[j,1]
    update_rect(j, bottom_left, height, width, is_out_box) 
    # orange/red out of circle
    # blue in circle
    
# for j in range(x.weight_a.shape[1]):
#     is_out_box = out_boxes[j]
#     bottom_left = (x.weight_a[0,j], x.weight_a[1,j])
#     width = (1-x.weight_a[2,j]) - x.weight_a[0,j]
#     height =(1-x.weight_a[3,j]) - x.weight_a[1,j]
#     update_rect(j, bottom_left, height, width, is_out_box)

# training_predictions = Counter()
# for i in range(num_pats):
#     test_input = a[:, i, np.newaxis]
#     ground_truth = bmat[:, i, np.newaxis]
#     complement_encoded_input = complement_encode(test_input)        
#     prediction = x.predict(complement_encoded_input)
#     correct = np.all(prediction == ground_truth)
#     training_predictions.update([correct])
# print(training_predictions)